# RAW DATA ecom_orders

In [0]:
%sql
SELECT * FROM `cohort_analysis_biqquery_catalog`.`cohort_db`.`ecom_orders`LIMIT 10;

row_id,customer_id,order_date,order_id,sales
1,CUST001,2024-06-02,ORD1000,76.93
2,CUST001,2024-07-26,ORD1001,251.35
3,CUST001,2024-10-12,ORD1002,135.91
797,CUST001,2024-11-15,ORD1796,15.89
973,CUST001,2024-04-12,ORD1972,295.76
4,CUST002,2024-04-11,ORD1003,217.41
5,CUST002,2024-05-24,ORD1004,291.87
887,CUST002,2024-07-22,ORD1886,272.75
6,CUST003,2024-05-28,ORD1005,23.09
7,CUST003,2024-07-18,ORD1006,134.71


# Step 1: First purchase date

In [0]:
CREATE OR REPLACE TEMP VIEW first_purchase AS
SELECT
  customer_id,
  MIN(order_date) AS first_purchase_date
FROM `cohort_analysis_biqquery_catalog`.`cohort_db`.`ecom_orders`
GROUP BY customer_id;


#  Step 2: Second purchase **date**

In [0]:
CREATE OR REPLACE TEMP VIEW second_purchase AS
SELECT
  o.customer_id,
  MIN(o.order_date) AS second_purchase_date
FROM `cohort_analysis_biqquery_catalog`.`cohort_db`.`ecom_orders` o
JOIN first_purchase f ON o.customer_id = f.customer_id
WHERE o.order_date > f.first_purchase_date
GROUP BY o.customer_id;

# Step 3: Third purchase date

In [0]:
CREATE OR REPLACE TEMP VIEW third_purchase AS
SELECT
  o.customer_id,
  MIN(o.order_date) AS third_purchase_date
FROM `cohort_analysis_biqquery_catalog`.`cohort_db`.`ecom_orders` o
JOIN second_purchase s ON o.customer_id = s.customer_id
WHERE o.order_date > s.second_purchase_date
GROUP BY o.customer_id;


# Step 4: Fourth purchase date

In [0]:
CREATE OR REPLACE TEMP VIEW fourth_purchase AS
SELECT
  o.customer_id,
  MIN(o.order_date) AS fourth_purchase_date
FROM `cohort_analysis_biqquery_catalog`.`cohort_db`.`ecom_orders` o
JOIN third_purchase s ON o.customer_id = s.customer_id
WHERE o.order_date > s.third_purchase_date
GROUP BY o.customer_id;

# Step 5: Show Schemas in Workspace

In [0]:
SHOW SCHEMAS IN workspace;

databaseName
default
information_schema


# Step 6: Save as Final Table

In [0]:
CREATE OR REPLACE TEMP VIEW customer_cohorts AS
SELECT
  f.customer_id,
  f.first_purchase_date,
  s.second_purchase_date,
  t.third_purchase_date,
  fp.fourth_purchase_date,
  DATEDIFF(s.second_purchase_date, f.first_purchase_date) AS days_between_1_and_2,
  DATEDIFF(t.third_purchase_date, s.second_purchase_date) AS days_between_2_and_3,
  DATEDIFF(fp.fourth_purchase_date, t.third_purchase_date) AS days_between_3_and_4,
  DATE_TRUNC('month', f.first_purchase_date) AS cohort_month
FROM first_purchase f
LEFT JOIN second_purchase s ON f.customer_id = s.customer_id
LEFT JOIN third_purchase t ON f.customer_id = t.customer_id
LEFT JOIN fourth_purchase fp ON f.customer_id = fp.customer_id;


# CHECK

In [0]:
CREATE OR REPLACE TABLE cohort_analysis AS
SELECT * FROM customer_cohorts;

num_affected_rows,num_inserted_rows


In [0]:
SELECT * FROM cohort_analysis ORDER BY first_purchase_date LIMIT 10;

customer_id,first_purchase_date,second_purchase_date,third_purchase_date,fourth_purchase_date,days_between_1_and_2,days_between_2_and_3,days_between_3_and_4,cohort_month
CUST038,2024-01-01,2024-03-27,2024-05-07,2024-05-29,86,41,22,2024-01-01T00:00:00.000Z
CUST030,2024-01-01,2024-01-27,2024-03-21,2024-08-16,26,54,148,2024-01-01T00:00:00.000Z
CUST072,2024-01-02,2024-02-07,2024-02-19,2024-03-24,36,12,34,2024-01-01T00:00:00.000Z
CUST137,2024-01-02,2024-02-18,2024-02-26,null,47,8,null,2024-01-01T00:00:00.000Z
CUST192,2024-01-02,2024-01-24,2024-03-13,2024-11-25,22,49,257,2024-01-01T00:00:00.000Z
CUST089,2024-01-03,2024-02-28,2024-03-07,2024-04-29,56,8,53,2024-01-01T00:00:00.000Z
CUST065,2024-01-03,2024-05-30,2024-06-29,2024-06-30,148,30,1,2024-01-01T00:00:00.000Z
CUST157,2024-01-03,2024-02-04,2024-02-12,2024-04-29,32,8,77,2024-01-01T00:00:00.000Z
CUST181,2024-01-03,2024-02-11,2024-03-07,2024-04-07,39,25,31,2024-01-01T00:00:00.000Z
CUST104,2024-01-04,2024-01-23,2024-02-21,2024-04-29,19,29,68,2024-01-01T00:00:00.000Z


# Four Purchases

In [0]:
WITH retention_with_cohort AS (
  SELECT
    customer_id,
    DATE_FORMAT(DATE_TRUNC('month', first_purchase_date), 'yyyy-MM') AS cohort_month,
    FLOOR(MONTHS_BETWEEN(second_purchase_date, first_purchase_date)) AS months_diff_2,
    FLOOR(MONTHS_BETWEEN(third_purchase_date, first_purchase_date)) AS months_diff_3,
    FLOOR(MONTHS_BETWEEN(fourth_purchase_date, first_purchase_date)) AS months_diff_4
  FROM workspace.default.cohort_analysis
),

cohort_sizes AS (
  SELECT
    DATE_FORMAT(DATE_TRUNC('month', first_purchase_date), 'yyyy-MM') AS cohort_month,
    COUNT(DISTINCT customer_id) AS cohort_size
  FROM workspace.default.cohort_analysis
  GROUP BY 1
),

retention_rates AS (
  SELECT
    r.cohort_month,

    -- Обернули cohort_size в ANY_VALUE
    ROUND(COUNT(CASE WHEN months_diff_2 = 0 THEN 1 END) * 1.0 / ANY_VALUE(c.cohort_size), 2) AS retention_2nd_m0,
    ROUND(COUNT(CASE WHEN months_diff_2 = 1 THEN 1 END) * 1.0 / ANY_VALUE(c.cohort_size), 2) AS retention_2nd_m1,
    ROUND(COUNT(CASE WHEN months_diff_3 = 1 THEN 1 END) * 1.0 / ANY_VALUE(c.cohort_size), 2) AS retention_3rd_m1,
    ROUND(COUNT(CASE WHEN months_diff_4 = 2 THEN 1 END) * 1.0 / ANY_VALUE(c.cohort_size), 2) AS retention_4th_m2

  FROM retention_with_cohort r
  JOIN cohort_sizes c ON r.cohort_month = c.cohort_month
  GROUP BY r.cohort_month
)

SELECT * FROM retention_rates
ORDER BY cohort_month;


cohort_month,retention_2nd_m0,retention_2nd_m1,retention_3rd_m1,retention_4th_m2
2024-01,0.38,0.21,0.17,0.09
2024-02,0.35,0.29,0.23,0.06
2024-03,0.30,0.42,0.24,0.18
2024-04,0.48,0.41,0.30,0.15
2024-05,0.50,0.44,0.50,0.25
2024-06,0.30,0.70,0.30,0.00


# 📊 Retention Rate by Cohort

In [0]:
WITH retention_data AS (
  SELECT
    customer_id,
    DATE_TRUNC('month', first_purchase_date) AS cohort_month,
    FLOOR(MONTHS_BETWEEN(second_purchase_date, first_purchase_date)) AS month_diff
  FROM workspace.default.cohort_analysis
  WHERE second_purchase_date IS NOT NULL
),

cohort_sizes AS (
  SELECT
    DATE_TRUNC('month', first_purchase_date) AS cohort_month,
    COUNT(DISTINCT customer_id) AS cohort_size
  FROM workspace.default.cohort_analysis
  GROUP BY 1
),

retention_rates AS (
  SELECT
    r.cohort_month,
    COUNT(DISTINCT CASE WHEN month_diff = 1 THEN customer_id END) * 1.0 / cs.cohort_size AS retention_rate_1m,
    COUNT(DISTINCT CASE WHEN month_diff = 2 THEN customer_id END) * 1.0 / cs.cohort_size AS retention_rate_2m,
    COUNT(DISTINCT CASE WHEN month_diff = 3 THEN customer_id END) * 1.0 / cs.cohort_size AS retention_rate_3m
  FROM retention_data r
  JOIN cohort_sizes cs ON r.cohort_month = cs.cohort_month
  GROUP BY r.cohort_month, cs.cohort_size
)

SELECT
  DATE_FORMAT(cohort_month, 'yyyy-MM') AS cohort_month,
  ROUND(retention_rate_1m, 2) AS retention_rate_1m,
  ROUND(retention_rate_2m, 2) AS retention_rate_2m,
  ROUND(retention_rate_3m, 2) AS retention_rate_3m
FROM retention_rates
ORDER BY cohort_month;


cohort_month,retention_rate_1m,retention_rate_2m,retention_rate_3m
2024-01,0.21,0.18,0.06
2024-02,0.29,0.15,0.06
2024-03,0.42,0.15,0.12
2024-04,0.41,0.04,0.00
2024-05,0.44,0.06,0.00
2024-06,0.70,0.00,0.00


Databricks visualization. Run in Databricks to view.

| Cohort Month | Retention 1m | Retention 2m | Retention 3m | Interpretation                                                                                                                      |
| ------------ | ------------ | ------------ | ------------ | ----------------------------------------------------------------------------------------------------------------------------------- |
| **January**  | 38%          | 21%          | 18%          | Of the users who made their first purchase in January, 38% returned in the next month. Retention dropped to 18% by month 3.         |
| **February** | 35%          | 29%          | 15%          | Slightly stronger 2-month retention than January. Indicates better mid-term engagement.                                             |
| **March**    | 30%          | 42%          | 15%          | Unusual pattern: higher retention in month 2 than in month 1. May suggest delayed activation or data lag.                           |
| **April**    | 48%          | 41%          | 4%           | Strong early retention, but sharp drop-off by month 3. Most users churned after the second month.                                   |
| **May**      | 50%          | 44%          | 6%           | Excellent 1–2 month retention, but long-term engagement is weak. Follow-up strategy needed.                                         |
| **June**     | 30%          | 70% ⚠️       | 0% ⚠️        | ⚠️ Data for June is still incomplete. 2m and 3m retention values may be artificially inflated or unavailable. Interpret cautiously. |


Based on the updated chart and data you provided, here are the **Key Insights** in **English**, written clearly and accurately:

---

### 🔍 **Key Insights: Retention Rate by Cohort**

1. **January Cohort (2024-01)**

   * **1-month retention:** 21%
   * **2-month retention:** 18%
   * **3-month retention:** 6%
     ➤ Weak retention across all months, with steady drop-off. Indicates poor engagement beyond initial purchase.

2. **February Cohort (2024-02)**

   * **1-month retention:** 29%
   * **2-month retention:** 15%
   * **3-month retention:** 6%
     ➤ Higher initial engagement than January, but similar long-term drop-off.

3. **March Cohort (2024-03)**

   * **1-month retention:** 42%
   * **2-month retention:** 15%
   * **3-month retention:** 12%
     ➤ Strong start, but rapid decrease afterward. Slight improvement over previous cohorts at 3 months.

4. **April Cohort (2024-04)**

   * **1-month retention:** 41%
   * **2-month retention:** 4%
   * **3-month retention:** 0%
     ➤ Very sharp decline after the first month. Possible product or experience-related churn issue.

5. **May Cohort (2024-05)**

   * **1-month retention:** 44%
   * **2-month retention:** 6%
   * **3-month retention:** 0%
     ➤ High initial retention, followed by strong drop-off. Similar pattern to April.

6. **June Cohort (2024-06)**

   * **1-month retention:** 70%
   * **2-month & 3-month retention:** 0% (data not yet available)
     ⚠️ **Caution:** June's long-term data is incomplete — full 2- and 3-month windows haven't elapsed. Still, initial retention looks promising.

---

### ✅ **Conclusion**

* The **March–May cohorts** show good initial engagement (40%+ retention in month 1), but **struggle to retain users long-term**.
* **June’s high 1-month rate (70%)** is promising, but early. Monitor closely.
* Priority: Improve **month 2+ experience**, identify **churn triggers**, and test **retention campaigns**.

If you'd like, I can help you turn this into a **slide or dashboard annotation**.


# Visualization 2: Repeat Purchase Rate by Cohort

In [0]:
WITH customer_orders AS (
  SELECT
    customer_id,
    DATE_FORMAT(DATE_TRUNC('month', first_purchase_date), 'yyyy-MM') AS cohort_month,
    CASE
      WHEN fourth_purchase_date IS NOT NULL THEN 4
      WHEN third_purchase_date IS NOT NULL THEN 3
      WHEN second_purchase_date IS NOT NULL THEN 2
      ELSE 1
    END AS total_orders
  FROM workspace.default.cohort_analysis
),

cohort_sizes AS (
  SELECT
    cohort_month,
    COUNT(*) AS cohort_size
  FROM customer_orders
  GROUP BY cohort_month
),

repeat_rates AS (
  SELECT
    c.cohort_month,
    ROUND(COUNT(CASE WHEN total_orders >= 2 THEN 1 END) * 1.0 / s.cohort_size, 2) AS repeat_rate_2nd_order,
    ROUND(COUNT(CASE WHEN total_orders >= 3 THEN 1 END) * 1.0 / s.cohort_size, 2) AS repeat_rate_3rd_order,
    ROUND(COUNT(CASE WHEN total_orders >= 4 THEN 1 END) * 1.0 / s.cohort_size, 2) AS repeat_rate_4th_order
  FROM customer_orders c
  JOIN cohort_sizes s ON c.cohort_month = s.cohort_month
  GROUP BY c.cohort_month, s.cohort_size
)

SELECT * FROM repeat_rates
ORDER BY cohort_month;


cohort_month,repeat_rate_2nd_order,repeat_rate_3rd_order,repeat_rate_4th_order
2024-01,1.00,0.98,0.83
2024-02,0.98,0.96,0.81
2024-03,1.00,0.94,0.82
2024-04,0.96,0.85,0.78
2024-05,1.00,0.94,0.69
2024-06,1.00,0.70,0.60


Databricks visualization. Run in Databricks to view.

### 🔍 **Key Insights – Repeat Purchase Rates by Cohort**

1. **Strong Second Purchase Rate**
   All cohorts show exceptionally high conversion to the 2nd purchase (between **96% and 100%**), indicating **strong customer engagement after the first order**.

2. **Declining Trend with Each Subsequent Purchase**
   There's a clear drop-off in repeat rates from 2nd → 3rd → 4th purchase across all cohorts, which is typical in e-commerce retention patterns.

3. **June 2024 Cohort Underperforms**
   The **June cohort (2024-06)** has the **lowest 4th order repeat rate at 60%**, compared to 83% in January. This may point to:

   * Lower quality of acquired users;
   * Changes in product, experience, or delivery;
   * Seasonality or campaign differences.

4. **March 2024 Cohort Performs Best Overall**
   With **1.00 (2nd order)**, **0.94 (3rd order)**, and **0.82 (4th order)**, this cohort demonstrates the most **balanced and sustainable repeat behavior**.


# Visualization 3: Cohort Size by Month


In [0]:
WITH first_orders AS (
  SELECT
    customer_id,
    DATE_TRUNC('month', first_purchase_date) AS cohort_month
  FROM workspace.default.cohort_analysis
  GROUP BY customer_id, first_purchase_date
)

SELECT
  DATE_FORMAT(cohort_month, 'yyyy-MM') AS cohort_month,
  COUNT(DISTINCT customer_id) AS new_customers
FROM first_orders
GROUP BY cohort_month
ORDER BY cohort_month;


cohort_month,new_customers
2024-01,66
2024-02,48
2024-03,33
2024-04,27
2024-05,16
2024-06,10


Databricks visualization. Run in Databricks to view.

### 🔍 **Key Insights: Cohort Size by Month**

1. January 2024 was the peak month for acquiring new customers, with a total of 66 new users — the largest cohort during the observed period.

2. Consistent decline: Each subsequent month shows a steady drop in cohort size, suggesting a downward trend in customer acquisition:

- February: 48
- March: 33
- April: 27
- May: 16
- June: 10

3. This decline could indicate:

- Reduced marketing efforts or lower ad spend
- Seasonal drop-off in interest or product relevance
- Potential user acquisition funnel issues (e.g., onboarding, outreach)

4. Actionable Insight: January’s acquisition strategy might hold valuable lessons — it’s worth analyzing what campaigns, channels, or offers were active then and if they can be replicated or scaled.